In [ ]:
import pandas as pd
import ast
import os

import matplotlib.pyplot as plt

import cartopy.crs as ccrs
import cartopy.feature as cfeature

import numpy as np

In [ ]:
directory_path = '../icao24_table'

# List all files in the directory
files = os.listdir(directory_path)

df_list = []

i = 0
# Iterate through the files and read each one
for file_name in files:
    if(file_name != '.DS_Store'):
        i = i+1
        print(file_name)
        print('{}/{}'.format(i, len(files)))
        file_path = os.path.join(directory_path, file_name)
        df_list.append(pd.read_csv(file_path))

In [ ]:
df = pd.concat(df_list, ignore_index=True)
df.drop(df.columns[0], axis=1, inplace = True)

# Replace '\n' with a comma in all columns
df['sensors'].replace('\n', ',', regex=True, inplace=True)
df['sensors'] = df['sensors'].apply(ast.literal_eval)

df

In [ ]:
# df.drop_duplicates(subset=['rawmsg'], inplace = True)
df.dropna(subset=['lat', 'lon'], inplace = True)

In [ ]:
plt.plot(df[df['icao24'] == 440018]['lon'], df[df['icao24'] == 440018]['lat'], alpha = 0.6)
plt.plot(df[df['icao24'] == '4ca8e4']['lon'], df[df['icao24'] == '4ca8e4']['lat'], alpha = 0.6)
plt.plot(df[df['icao24'] == '3461ce']['lon'], df[df['icao24'] == '3461ce']['lat'], alpha = 0.6)


In [ ]:
serial_list = []

print(len(df['sensors']))
changed_once = False

for i in range(len(df['sensors'])):
    sensor_list_from_df = df['sensors'].iloc[i]

    for sensor in sensor_list_from_df:
        if(sensor['serial'] not in serial_list):
            serial_list.append(sensor['serial'])

In [ ]:
df_receiver = pd.read_json('sensor_list.json')
df_receiver['lat'] = df_receiver['position'].apply(lambda x: x['latitude'])
df_receiver['lon'] = df_receiver['position'].apply(lambda x: x['longitude'])

In [ ]:
df_receiver = df_receiver[df_receiver['serial'].isin(serial_list)]
df_receiver = df_receiver[(df_receiver['lon'] > -30) & (df_receiver['lon'] < 30)]

In [ ]:
df_receiver

In [ ]:
plt.plot(df[df['icao24'] == 440018]['lon'], df[df['icao24'] == 440018]['lat'], alpha = 0.6)
plt.plot(df[df['icao24'] == '4ca8e4']['lon'], df[df['icao24'] == '4ca8e4']['lat'], alpha = 0.6)
plt.plot(df[df['icao24'] == '3461ce']['lon'], df[df['icao24'] == '3461ce']['lat'], alpha = 0.6)
plt.scatter(df_receiver['lon'], df_receiver['lat'], alpha = 0.6)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cartopy
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

ax = plt.axes(projection=ccrs.PlateCarree())
ax.add_feature(cartopy.feature.COASTLINE, linestyle='-', alpha=0.7)
ax.add_feature(cartopy.feature.BORDERS, linestyle='-', alpha=0.7)
# ax.set_extent([-25, 40, 35, 70])  # Adjust the extent to focus on Europe

shpfilename = shpreader.natural_earth(resolution='10m',
                                      category='cultural',
                                      name='admin_0_countries')
reader = shpreader.Reader(shpfilename)
countries = reader.records()

# gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, color='gray', alpha=0.3, linestyle='--')
# gl.right_labels = False
# gl.top_labels = False
# gl.xlines = False
# gl.ylines = False
# gl.xlocator = mticker.FixedLocator(range(-180, 181, 10))
# gl.ylocator = mticker.FixedLocator(range(-90, 91, 10))
# gl.xformatter = LONGITUDE_FORMATTER
# gl.yformatter = LATITUDE_FORMATTER
# gl.xlabel_style = {'size': 15, 'color': 'gray'}
# gl.xlabel_style = {'color': 'red', 'weight': 'bold'}

plt.plot(df[df['icao24'] == 440018]['lon'], df[df['icao24'] == 440018]['lat'], alpha = 1.0, color = 'red')
plt.plot(df[df['icao24'] == '4ca8e4']['lon'], df[df['icao24'] == '4ca8e4']['lat'], alpha = 1.0, color = 'orange')
plt.plot(df[df['icao24'] == '3461ce']['lon'], df[df['icao24'] == '3461ce']['lat'], alpha = 1.0, color = 'yellow')

# plt.scatter(df_receiver['lon'], df_receiver['lat'], s = 90**2, color = 'khaki', alpha = 1.0)
plt.scatter(df_receiver['lon'], df_receiver['lat'], linewidth = 0.6, alpha = 0.8, color = 'black')

# ax.text(-0.1, 0.5, 'Latitude', va='bottom', ha='center',
#         rotation='vertical', rotation_mode='anchor',
#         transform=ax.transAxes)
# ax.text(0.5, -0.1, 'Longitude', va='bottom', ha='center',
#         rotation='horizontal', rotation_mode='anchor',
#         transform=ax.transAxes)

# plt.title('Tracked flight for jitter identification')

plt.axis('square')

plt.show()


## Get closest receiver for each data point

In [ ]:
def kwikdist(lata, lona, latb, lonb):
    """
    Quick and dirty dist [km]
    In:
        lat/lon, lat/lon vectors [deg]
    Out:
        dist vector [nm]
    """

    re      = 6371000.  # readius earth [m]
    dlat    = np.radians(latb - lata)
    dlon    = np.radians(((lonb - lona)+180)%360-180)
    cavelat = np.cos(np.radians(lata + latb) * 0.5)

    dangle  = np.sqrt(np.multiply(dlat, dlat) +
                      np.multiply(np.multiply(dlon, dlon),
                                  np.multiply(cavelat, cavelat)))

    dist    = re * dangle / 1000

    return dist

In [ ]:
#get_closest_receiver
def get_closest_receiver(x):
    global counter
    
    counter += 1
    
    print('{}/{}'.format(counter, max_counter))
    closest_dist = 100000
    closest_rec = -666
    mintime = -1

    for sensor in x:
        if(len(df_receiver[df_receiver['serial'] == sensor['serial']]['lat']) > 0):
            lat_rec = float(df_receiver[df_receiver['serial'] == sensor['serial']]['lat'])
            lon_rec = float(df_receiver[df_receiver['serial'] == sensor['serial']]['lon'])
                        
            lat_ac = df.iloc[counter]['lat']
            lon_ac = df.iloc[counter]['lon']

            dist = kwikdist(lat_rec, lon_rec, lat_ac, lon_ac)

            if(dist < closest_dist):
                closest_dist = dist
                closest_rec = sensor['serial']
                mintime = sensor['mintime']

    return closest_rec, closest_dist, mintime

In [ ]:
global counter
counter = -1
max_counter = len(df['sensors'])

df[['cl_rec', 'cl_dist', 'cl_mintime']] = df['sensors'].apply(lambda x: pd.Series(get_closest_receiver(x)))

In [ ]:
df_close = df[df['cl_dist'] < 10000]
print(max(df_close['cl_dist']))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cartopy
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

ax = plt.axes(projection=ccrs.PlateCarree())
ax.add_feature(cartopy.feature.COASTLINE, linestyle='-', alpha=0.7)
ax.add_feature(cartopy.feature.BORDERS, linestyle='-', alpha=0.7)
# ax.set_extent([-25, 40, 35, 70])  # Adjust the extent to focus on Europe

shpfilename = shpreader.natural_earth(resolution='10m',
                                      category='cultural',
                                      name='admin_0_countries')
reader = shpreader.Reader(shpfilename)
countries = reader.records()

df_close = df.iloc[0:300]
# gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, color='gray', alpha=0.3, linestyle='--')
# gl.right_labels = False
# gl.top_labels = False
# gl.xlines = False
# gl.ylines = False
# gl.xlocator = mticker.FixedLocator(range(-180, 181, 10))
# gl.ylocator = mticker.FixedLocator(range(-90, 91, 10))
# gl.xformatter = LONGITUDE_FORMATTER
# gl.yformatter = LATITUDE_FORMATTER
# gl.xlabel_style = {'size': 15, 'color': 'gray'}
# gl.xlabel_style = {'color': 'red', 'weight': 'bold'}

plt.plot(df_close[df_close['icao24'] == 440018]['lon'], df_close[df_close['icao24'] == 440018]['lat'], alpha = 1.0, color = 'red')
plt.plot(df_close[df_close['icao24'] == '4ca8e4']['lon'], df_close[df_close['icao24'] == '4ca8e4']['lat'], alpha = 1.0, color = 'orange')
plt.plot(df_close[df_close['icao24'] == '3461ce']['lon'], df_close[df_close['icao24'] == '3461ce']['lat'], alpha = 1.0, color = 'yellow')

# plt.scatter(df_receiver['lon'], df_receiver['lat'], s = 90**2, color = 'khaki', alpha = 1.0)
plt.scatter(df_receiver['lon'], df_receiver['lat'], linewidth = 0.6, alpha = 0.8, color = 'black')

# ax.text(-0.1, 0.5, 'Latitude', va='bottom', ha='center',
#         rotation='vertical', rotation_mode='anchor',
#         transform=ax.transAxes)
# ax.text(0.5, -0.1, 'Longitude', va='bottom', ha='center',
#         rotation='horizontal', rotation_mode='anchor',
#         transform=ax.transAxes)

# plt.title('Tracked flight for jitter identification')

plt.axis('square')

plt.show()


In [ ]:
df = df[df['cl_mintime'] > 1]
df

In [ ]:
icao24_list = list(df['icao24'].unique())
df_icao24 = []

for ic in icao24_list:
    df_icao24.append(df[df['icao24'] == ic])

In [ ]:
update_interval_here = []

for df_local in df_icao24:
    rec_list = list(df_local['cl_rec'].unique())
    
    prev_mintime = 1000
    prev_cl_rec = 'here'
    
    for rec in rec_list:
        df_rec = df_local[df_local['cl_rec'] == rec]
        
        
        for index, row in df_rec.iterrows():
            curr_mintime = row.cl_mintime
            curr_cl_rec = row.cl_rec
            
            diff = curr_mintime - prev_mintime
            print(diff)
            
            prev_mintime = curr_mintime
            prev_cl_rec = row.cl_rec
            
            if(diff < 0.75):
                update_interval_here.append(diff)
    
update_interval_here
# update_interval_here = [item for sublist in update_interval_here for item in sublist]

In [ ]:
update_interval_here = np.array(update_interval_here)
update_interval_here = update_interval_here[update_interval_here < 2]

In [ ]:
start = 0.25
end = 0.75
nb_of_bar = 40

input_list = np.array(df['updateinterval'].to_list())

bins = np.linspace(start, end, int(((end-start)*nb_of_bar+1)))
weightsa = np.ones_like(input_list)/float(len(input_list))
freq_, edge_ = np.histogram(input_list, bins, weights = weightsa)

within_5s = sum(freq_[(edge_[:-1] <= 0.75) & (edge_[:-1] >= 0.25)])*100
print(within_5s)

plt.bar(edge_[:-1], freq_*100, width = 1/nb_of_bar, edgecolor = 'k')
plt.xlabel('Time [s]')
plt.ylabel('Reception Probability [%]')

In [ ]:
edge_perfect = []
probability_perfect = []
prob = 0.8

for i in range(5):
    edge_perfect.append(0.5 * (i+1))
    
    prob_now = prob*(1-prob)**i
    
    probability_perfect.append(prob_now)
    
probability_perfect = np.array(probability_perfect)
edge_perfect = np.array(edge_perfect)
print(probability_perfect)

plt.bar(edge_perfect, probability_perfect*100, width = 1/10, edgecolor = 'k')
plt.xlabel('Time [s]')
plt.ylabel('Frequency [%]')

In [ ]:
start = 0.0
end = 4.0
nb_of_bar = 20

input_list = df['updateinterval'].to_list()

bins = np.linspace(start, end, int(((end-start)*nb_of_bar+1)))
weightsa = np.ones_like(input_list)/float(len(input_list))
freq_, edge_ = np.histogram(input_list, bins, weights = weightsa)

within_5s = sum(freq_[(edge_[:-1] <= 0.75) & (edge_[:-1] >= 0.25)])*100
print(within_5s)

plt.bar(edge_[:-1], freq_*100, width = 1/nb_of_bar, edgecolor = 'k')
plt.xlabel('Time [s]')
plt.ylabel('Reception Probability [%]')

## How to calculate only between the same receiver if consecutive

In [ ]:
# icao_list = [440018, '3461ce', '4ca8e4']

# df_new = []

# for icao in icao_list:
#     df_ = df[df['icao24'] == icao]

#     df_grouped = df_.groupby('cl_rec', group_keys=True).apply(lambda x: x)
#     df_grouped['updateinterval'] = df_grouped['cl_mintime'].diff()

#     df_grouped = df_grouped[(df_grouped['updateinterval'] > 0.0) & (df_grouped['updateinterval'] < 10)]
#     df_ = df_grouped.reset_index(drop=True)
    
#     df_new.append(df_)
    
# df_new = pd.concat(df_new)

In [ ]:
df_grouped = df.groupby('icao24', group_keys=True).apply(lambda x: x)
df_grouped['updateinterval'] = df_grouped['mintime'].diff()

df_grouped = df_grouped[(df_grouped['updateinterval'] > 0.0) & (df_grouped['updateinterval'] < 2)]
df = df_grouped.reset_index(drop=True)

In [ ]:
start = 0.0
end = 2.0
nb_of_bar = 20

input_list = df['updateinterval'].to_list()

bins = np.linspace(start, end, int(((end-start)*nb_of_bar+1)))
weightsa = np.ones_like(input_list)/float(len(input_list))
freq_, edge_ = np.histogram(input_list, bins, weights = weightsa)

within_5s = sum(freq_[(edge_[:-1] <= 0.75) & (edge_[:-1] >= 0.25)])*100
within_next_5s = sum(freq_[(edge_[:-1] <= 1.25) & (edge_[:-1] >= 0.75)])*100
within_next_next_5s = sum(freq_[(edge_[:-1] <= 1.75) & (edge_[:-1] >= 1.25)])*100

print(within_5s, within_next_5s, within_next_next_5s, (within_5s + within_next_5s + within_next_next_5s))

plt.bar(edge_[:-1], freq_*100, width = 1/nb_of_bar, edgecolor = 'k')
plt.xlabel('Time [s]')
plt.ylabel('Frequency [%]')

In [ ]:
here1 = sum(freq_[edge_[:-1] < 0.25]*100)
here2 = sum(freq_[(edge_[:-1] > 0.25) & (edge_[:-1] < 0.75)]*100)
here3 = sum(freq_[(edge_[:-1] > 0.75) & (edge_[:-1] < 1.25)]*100)

print(here1, here2, here3)
print(here1+here2+here3)

In [ ]:
100-24.8-68.6

In [ ]:
start = 0.0
end = 2.0
nb_of_bar = 50

df_here = df[(df['updateinterval'] > start) & (df['updateinterval'] < end)]
input_list = df_here['updateinterval'].to_list()

bins = np.linspace(start, end, int(((end-start)*nb_of_bar+1)))
weightsa = np.ones_like(input_list)/float(len(input_list))
freq_, edge_ = np.histogram(input_list, bins, weights = weightsa)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

axes[0].bar(edge_[:-1], freq_*100, width = 1/nb_of_bar, edgecolor = 'k')
axes[0].set_xlabel('Time [s]')
axes[0].set_ylabel('Frequency [%]')

start = 0.25
end = 0.75
nb_of_bar = 50

input_list = df['updateinterval'].to_list()

bins = np.linspace(start, end, int(((end-start)*nb_of_bar+1)))
weightsa = np.ones_like(input_list)/float(len(input_list))
freq_, edge_ = np.histogram(input_list, bins, weights = weightsa)

axes[1].bar(edge_[:-1], freq_*100, width = 1/nb_of_bar, edgecolor = 'k')
axes[1].set_xlabel('Time [s]')
# axes[1].ylabel('Reception Probability [%]')

fig.tight_layout()

within_5s = sum(freq_[(edge_[:-1] <= 0.75) & (edge_[:-1] >= 0.25)])*100
print(within_5s)

In [ ]:
def get_cum(freq):
    cum_list = []
    cum = 0
    
    for f in freq:
        cum += f
        cum_list.append(cum)
        
    return np.array(cum_list)

In [ ]:
cum_list = get_cum(freq_)
print(cum_list)

plt.plot(edge_[:-1], cum_list)

## Freq Analysis

In [ ]:
def get_freq_domain(y, Fs):
    n = len(y)                       # length of the signal
    k = np.arange(n)
    T = n/Fs
    frq = k/T # two sides frequency range
    freq = frq[range(int(n/2))]           # one side frequency range

    Y = np.fft.fft(y)/n              # fft computing and normalization
    Y = Y[range(int(n/2))]
    PSD = Y * np.conj(Y)

    return freq, Y

In [ ]:
start = 0.2
end = 0.8
clearance = 0.05
nb_of_bar = 50

df_here = df[(df['updateinterval'] > start + clearance) & (df['updateinterval'] < end - clearance)]
input_list = df_here['updateinterval'].to_list()

bins = np.linspace(start, end, int(((end-start)*nb_of_bar+1)))
weightsa = np.ones_like(input_list)/float(len(input_list))
freq_, edge_ = np.histogram(input_list, bins, weights = weightsa)

plt.bar(edge_[:-1], freq_*100, width = 1/nb_of_bar, edgecolor = 'k')
plt.xlabel('Time [s]')
plt.ylabel('Reception Probability [%]')

within_5s = sum(freq_[(edge_[:-1] <= 0.75) & (edge_[:-1] >= 0.25)])*100
print(within_5s)

In [ ]:
fq, Y = get_freq_domain(freq_, nb_of_bar)

plt.subplot(2,1,1)
plt.bar(edge_[:-1], freq_*100, width = 1/nb_of_bar, edgecolor = 'k')
plt.xlabel('time')
plt.ylabel('amplitude')

plt.subplot(2,1,2)

plt.plot(fq, abs(Y), 'r-')
plt.xlabel('freq (Hz)')
plt.ylabel('|Y(freq)|')
plt.xlim([0, 20])